# 02_data_modeling.ipynb

## Invoice and Line Item Extraction Summary

**Goal:**
Transform the cleaned OCR invoice data (`regex_cleaned_invoices.csv`) into two structured tables for easier downstream processing.

**Output Files:**

* `regex_invoices.csv` → one row per invoice
  **Columns:** `file_path`, `invoice_id`, `vendor`, `date`, `total`, `invoice_number`
* `regex_lineitems.csv` → one row per product line
  **Columns:** `file_path`, `invoice_id`, `description`, `quantity`, `unit_price`, `total`

**Purpose:**

1. Separate invoice-level and item-level data for normalization.
2. Enable consistent schema even when some invoices lack line item details.
3. Prepare structured data for later LLM-based enrichment or verification.


In [3]:
import os
import pandas as pd

# Define folder paths
root_folder = os.path.abspath(os.path.join(os.getcwd(), ".."))
processed_folder = os.path.join(root_folder, "data", "processed")

# Create folder if it doesn't exist
os.makedirs(processed_folder, exist_ok=True)

# Define output file paths
invoices_csv = os.path.join(processed_folder, "regex_invoices.csv")
lineitems_csv = os.path.join(processed_folder, "regex_lineitems.csv")

# Create empty CSVs with defined schema
invoices_cols = ["file_path", "invoice_id", "vendor", "date", "total", "invoice_number"]
lineitems_cols = ["file_path", "invoice_id", "description", "quantity", "unit_price", "total"]

pd.DataFrame(columns=invoices_cols).to_csv(invoices_csv, index=False)
pd.DataFrame(columns=lineitems_cols).to_csv(lineitems_csv, index=False)

print("Created empty CSV files:")
print(f"Invoices → {invoices_csv}")
print(f"LineItems → {lineitems_csv}")


Created empty CSV files:
Invoices → c:\Stealth AI\Clean Reader\data\processed\regex_invoices.csv
LineItems → c:\Stealth AI\Clean Reader\data\processed\regex_lineitems.csv


In [11]:
import os
import re
import pandas as pd

# Load cleaned invoice text table
df = pd.read_csv(r"..\data\processed\regex_cleaned_invoices.csv")

# Define output paths
root_folder = os.path.abspath(os.path.join(os.getcwd(), ".."))
processed_folder = os.path.join(root_folder, "data", "processed", "regex")
os.makedirs(processed_folder, exist_ok=True)

invoices_csv = os.path.join(processed_folder, "regex_invoices.csv")
lineitems_csv = os.path.join(processed_folder, "regex_lineitems.csv")

invoices_data = []
lineitems_data = []

# Clean description function
def clean_description(desc: str) -> str:
    desc = desc.lower().strip()
    desc = re.sub(r"\b\d+\s*[x#]\s*\d*\s*(oz|lb|cs|ea|bn|bag)?\b", "", desc)
    desc = re.sub(r"\b(cs|ea|bn|bag|lb|oz|#)\b", "", desc)
    desc = re.sub(r"[^a-z0-9\s,]+", " ", desc)
    desc = re.sub(r"\s+", " ", desc)
    return desc.strip()

# Loop through each invoice entry
for _, row in df.iterrows():
    text = str(row.get("cleaned_text", ""))
    file_path = row.get("file_path", "")
    invoice_id = int(row.get("id", 0)) if not pd.isna(row.get("id", 0)) else 0
    vendor = row.get("vendor", "")
    date = row.get("date", "")
    total = row.get("total", "")
    invoice_number = int(row.get("invoice_number", 0)) if not pd.isna(row.get("invoice_number", 0)) else 0

    # Invoice-level entry
    invoices_data.append({
        "file_path": file_path,
        "invoice_id": invoice_id,
        "vendor": vendor,
        "date": date,
        "total": total,
        "invoice_number": invoice_number
    })

    # Line item extraction
    line_pattern = re.compile(
        r"(\d{4,6})?\s*([\d\.]+)\s+(?:ea|cs|bn|bag|lb|oz|#)?\s*([A-Za-z0-9\s\+\-,#&/]+?)\s+(\d+\.\d{2})",
        re.IGNORECASE
    )

    matches = line_pattern.findall(text)
    if matches:
        for match in matches:
            _, qty, desc, total_price = match
            try:
                qty = int(float(qty))
            except:
                qty = 0
            total_price = total_price.strip()
            try:
                unit_price = round(float(total_price) / float(qty), 2) if qty != 0 else ""
            except:
                unit_price = ""
            clean_desc = clean_description(desc)
            lineitems_data.append({
                "file_path": file_path,
                "invoice_id": invoice_id,
                "description": clean_desc,
                "quantity": qty,
                "unit_price": unit_price,
                "total": total_price
            })
    else:
        # Add placeholder if no matches found
        lineitems_data.append({
            "file_path": file_path,
            "invoice_id": invoice_id,
            "description": "",
            "quantity": 0,
            "unit_price": "",
            "total": ""
        })

# Save extracted data
pd.DataFrame(invoices_data).to_csv(invoices_csv, index=False)
pd.DataFrame(lineitems_data).to_csv(lineitems_csv, index=False)

# Sanity Check Across All Processed Tables
train_dir = os.path.join(root_folder, "data", "raw", "train")
train_files = {os.path.basename(f).lower() for f in os.listdir(train_dir)}

def get_filenames_from_column(df, column_name="file_path"):
    return {os.path.basename(str(f)).lower() for f in df[column_name].dropna() if isinstance(f, str)}

cleaned_df = pd.read_csv(r"..\data\processed\regex_cleaned_invoices.csv")
invoices_df = pd.read_csv(invoices_csv)
lineitems_df = pd.read_csv(lineitems_csv)

cleaned_files = get_filenames_from_column(cleaned_df)
invoices_files = get_filenames_from_column(invoices_df)
lineitems_files = get_filenames_from_column(lineitems_df)

missing_in_cleaned = train_files - cleaned_files
missing_in_invoices = train_files - invoices_files
missing_in_lineitems = train_files - lineitems_files

print("Sanity Check: File Presence Across Datasets")
print(f"Total files in train folder: {len(train_files)}")
print(f"Missing in regex_cleaned_invoices.csv: {len(missing_in_cleaned)}")
print(f"Missing in regex_invoices.csv: {len(missing_in_invoices)}")
print(f"Missing in regex_lineitems.csv: {len(missing_in_lineitems)}")

if missing_in_cleaned:
    print("\nFiles missing from regex_cleaned_invoices.csv:")
    for f in sorted(missing_in_cleaned):
        print(f)

if missing_in_invoices:
    print("\nFiles missing from regex_invoices.csv:")
    for f in sorted(missing_in_invoices):
        print(f)

if missing_in_lineitems:
    print("\nFiles missing from regex_lineitems.csv:")
    for f in sorted(missing_in_lineitems):
        print(f)


Sanity Check: File Presence Across Datasets
Total files in train folder: 57
Missing in regex_cleaned_invoices.csv: 0
Missing in regex_invoices.csv: 0
Missing in regex_lineitems.csv: 0


In [14]:
import os
import pandas as pd

# Define paths
root_folder = os.path.abspath(os.path.join(os.getcwd(), ".."))
processed_folder = os.path.join(root_folder, "data", "processed")
regex_folder = os.path.join(root_folder, "data", "processed","regex")
train_folder = os.path.join(root_folder, "data", "raw", "train")

# Load all three processed datasets
cleaned_df = pd.read_csv(os.path.join(processed_folder, "regex_cleaned_invoices.csv"))
invoices_df = pd.read_csv(os.path.join(regex_folder, "regex_invoices.csv"))
lineitems_df = pd.read_csv(os.path.join(regex_folder, "regex_lineitems.csv"))

# Create file name sets (normalize for comparison)
train_files = {os.path.basename(f) for f in os.listdir(train_folder)}
cleaned_files = {os.path.basename(f) for f in cleaned_df["file_path"]}
invoices_files = {os.path.basename(f) for f in invoices_df["file_path"]}
lineitems_files = {os.path.basename(f) for f in lineitems_df["file_path"]}

# Identify missing file names per dataset
missing_in_cleaned = train_files - cleaned_files
missing_in_invoices = train_files - invoices_files
missing_in_lineitems = train_files - lineitems_files

print("Sanity Check: File Presence Across Datasets")
print(f"Total files in train: {len(train_files)}")
print(f"Missing in regex_cleaned_invoices.csv: {len(missing_in_cleaned)}")
print(f"Missing in regex_invoices.csv: {len(missing_in_invoices)}")
print(f"Missing in regex_lineitems.csv: {len(missing_in_lineitems)}")

if missing_in_cleaned:
    print("\nFiles missing from regex_cleaned_invoices.csv:")
    for f in sorted(missing_in_cleaned):
        print(f)

if missing_in_invoices:
    print("\nFiles missing from regex_invoices.csv:")
    for f in sorted(missing_in_invoices):
        print(f)

if missing_in_lineitems:
    print("\nFiles missing from regex_lineitems.csv:")
    for f in sorted(missing_in_lineitems):
        print(f)


Sanity Check: File Presence Across Datasets
Total files in train: 57
Missing in regex_cleaned_invoices.csv: 0
Missing in regex_invoices.csv: 0
Missing in regex_lineitems.csv: 0
